# Create pyCIAM Storm Costs Lookup Table

See the corresponding notebook in the pyCIAM github repo for more details. This is a time-consuming notebook that is not run in the code ocean capsule by default. Instead, we include the intermediate outputs as a pre-computed data file. You may run this notebook to reproduce this step if desired.

## Setup

In [4]:
import distributed as dd
import pandas as pd
from pyCIAM.surge import damage_funcs
from pyCIAM.surge.lookup import create_surge_lookup
from shared import (
    PATH_PARAMS,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    start_dask_cluster,
)

In [9]:
SEG_CHUNKSIZE = 50

PARAMS = pd.read_json(PATH_PARAMS)["values"]

In [10]:
DMF_I = getattr(damage_funcs, PARAMS.dmf + "_i")
DDF_I = getattr(damage_funcs, PARAMS.ddf + "_i")

In [ ]:
client, cluster = start_dask_cluster()
cluster

## Run surge damage calculations for each combo

In [11]:
futs = {}
for kind, sliiders in [("seg_adm", PATH_SLIIDERS), ("seg", PATH_SLIIDERS_SEG)]:
    futs[kind] = create_surge_lookup(
        sliiders,
        [PATH_SLR_INT],
        PATHS_SURGE_LOOKUP[kind],
        kind,
        PARAMS.at_start,
        PARAMS.n_interp_pts_lslr,
        PARAMS.n_interp_pts_rhdiff,
        DDF_I,
        DMF_I,
        quantiles=[0.5],
        start_year=PARAMS.model_start,
        slr_0_years=PARAMS.slr_0_year,
        client=client,
        force_overwrite=True,
        seg_chunksize=SEG_CHUNKSIZE,
        mc_dim="quantile",
    )

## Close

In [ ]:
# ensure completion and close cluster
all_futs = futs["seg"] + futs["seg_adm"]
dd.wait(all_futs)
assert [f.status == "finished" for f in all_futs]
finished = True

In [ ]:
client.cluster.close(), client.close()